#  Task 2 – Trial Store Performance Analysis
This notebook contains the solution for Task 2 of the Quantium Virtual Internship. The goal is to assess the impact of a trial period in selected stores by comparing performance with matched control stores using statistical and visual techniques.

We will:
- Identify suitable control stores based on sales & customer similarity
- Visually and statistically assess the impact of trials for stores 77, 86, and 88
- Conclude the effectiveness of the trials


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset prepared in Task 1
data = pd.read_csv("Data/processed/QVI_data.csv")



In [56]:
# Convert DATE to datetime and create YEARMONTH
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEARMONTH'] = data['DATE'].dt.to_period('M').astype(str).str.replace("-", "").astype(int)

In [57]:
# Aggregate monthly metrics
monthly_metrics = data.groupby(['STORE_NBR', 'YEARMONTH']).agg(
    totSales=('TOT_SALES', 'sum'),
    nCustomers=('LYLTY_CARD_NBR', pd.Series.nunique),
    nTxn=('TXN_ID', 'nunique'),
    totalQty=('PROD_QTY', 'sum')
).reset_index()


In [58]:
# Derived metrics
monthly_metrics['nTxnPerCust'] = monthly_metrics['nTxn'] / monthly_metrics['nCustomers']
monthly_metrics['nChipsPerTxn'] = monthly_metrics['totalQty'] / monthly_metrics['nTxn']
monthly_metrics['avgPricePerUnit'] = monthly_metrics['totSales'] / monthly_metrics['totalQty']
monthly_metrics.drop(columns=['nTxn', 'totalQty'], inplace=True)


In [59]:
# Filter pre-trial data (July 2018 – Dec 2018)
pre_trial = monthly_metrics[(monthly_metrics['YEARMONTH'] >= 201807) & (monthly_metrics['YEARMONTH'] <= 201812)]
store_counts = pre_trial.groupby('STORE_NBR')['YEARMONTH'].count()
valid_stores = store_counts[store_counts == 6].index.tolist()
pre_trial = pre_trial[pre_trial['STORE_NBR'].isin(valid_stores)]

In [60]:
# Define correlation function
def calculate_correlation(input_df, metric_col, trial_store):
    trial_data = input_df[input_df['STORE_NBR'] == trial_store][['YEARMONTH', metric_col]]
    output = []

    for store in input_df['STORE_NBR'].unique():
        if store == trial_store:
            continue
        control_data = input_df[input_df['STORE_NBR'] == store][['YEARMONTH', metric_col]]
        merged = pd.merge(trial_data, control_data, on='YEARMONTH', suffixes=('_trial', '_control'))
        if not merged.empty and metric_col + '_trial' in merged.columns and metric_col + '_control' in merged.columns:
            corr = merged[metric_col + '_trial'].corr(merged[metric_col + '_control'])
            output.append({'Store1': trial_store, 'Store2': store, 'corr_measure': corr})

    return pd.DataFrame(output)


In [61]:




# Define magnitude distance function
def calculate_magnitude_distance(input_df, metric_col, trial_store):
    results = []
    trial_vals = input_df[input_df['STORE_NBR'] == trial_store][metric_col].values

    for store in input_df['STORE_NBR'].unique():
        if store == trial_store:
            continue
        control_vals = input_df[input_df['STORE_NBR'] == store][metric_col].values
        if len(control_vals) != len(trial_vals):
            continue
        abs_diff = np.abs(trial_vals - control_vals)
        if np.max(abs_diff) - np.min(abs_diff) == 0:
            normalized = np.ones_like(abs_diff)
        else:
            normalized = 1 - (abs_diff - np.min(abs_diff)) / (np.max(abs_diff) - np.min(abs_diff))

        results.append({'Store1': trial_store, 'Store2': store, 'mag_measure': np.mean(normalized)})

    return pd.DataFrame(results)



In [62]:
# Choose a valid trial store from the list
print("Valid stores:", valid_stores)
trial_store = valid_stores[0]  # Or choose manually

# Run correlation and magnitude comparisons
corr_sales = calculate_correlation(pre_trial, 'totSales', trial_store)
mag_sales = calculate_magnitude_distance(pre_trial, 'totSales', trial_store)

# Merge and score
if not corr_sales.empty and not mag_sales.empty:
    sales_score = pd.merge(corr_sales, mag_sales, on=['Store1', 'Store2'])
    sales_score['score'] = sales_score[['corr_measure', 'mag_measure']].mean(axis=1)
    top_matches = sales_score.sort_values(by='score', ascending=False)
    print("Top control store matches:")
    print(top_matches.head())
else:
    print("❌ No valid correlation or magnitude results. Check your trial store and data coverage.")


Valid stores: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 22

In [64]:
from scipy.stats import ttest_ind

def evaluate_trial_impact(monthly_metrics, trial_store, control_store, metric='totSales'):
    # Filter for trial and control stores during trial period
    trial_period = [201903, 201904, 201905]
    
    trial_data = monthly_metrics[
        (monthly_metrics['STORE_NBR'] == trial_store) &
        (monthly_metrics['YEARMONTH'].isin(trial_period))
    ][metric]

    control_data = monthly_metrics[
        (monthly_metrics['STORE_NBR'] == control_store) &
        (monthly_metrics['YEARMONTH'].isin(trial_period))
    ][metric]

    # Perform independent t-test
    t_stat, p_val = ttest_ind(trial_data, control_data, equal_var=False)

    print(f"\nTrial Store: {trial_store} vs Control Store: {control_store}")
    print(f"Metric: {metric}")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_val:.6f}")
    if p_val < 0.05:
        print("✅ Significant difference detected during trial period.")
    else:
        print("❌ No significant difference detected during trial period.")

evaluate_trial_impact(monthly_metrics, 1, 204)
evaluate_trial_impact(monthly_metrics, 1, 159)
evaluate_trial_impact(monthly_metrics, 1, 161)



Trial Store: 1 vs Control Store: 204
Metric: totSales
T-statistic: 11.3458
P-value: 0.000359
✅ Significant difference detected during trial period.

Trial Store: 1 vs Control Store: 159
Metric: totSales
T-statistic: 11.3558
P-value: 0.000397
✅ Significant difference detected during trial period.

Trial Store: 1 vs Control Store: 161
Metric: totSales
T-statistic: 16.1883
P-value: 0.000245
✅ Significant difference detected during trial period.


##  Trial Impact Summary – Store 1

We compared **Trial Store 1** against its top control store candidates based on 6-month pre-trial similarity in sales trends and magnitude.  
We then performed t-tests on **total sales** during the trial period (Mar–May 2019) to evaluate statistical significance.

###  T-Test Results

| Trial Store | Control Store | T-statistic | P-value   | Result                     |
|-------------|----------------|-------------|-----------|----------------------------|
| 1           | 204            | 11.3458     | 0.000359  | ✅ Significant difference   |
| 1           | 159            | 11.3558     | 0.000397  | ✅ Significant difference   |
| 1           | 161            | 16.1883     | 0.000245  | ✅ Significant difference   |

### Interpretation:

- All top control matches for Store 1 show **statistically significant uplift** in sales during the trial period.
- This strongly supports the hypothesis that the **trial positively influenced sales performance** at Store 1.
- Control store **204** remains a strong match and can be used as the **primary comparison store** in final presentations.

---

## Recommendation:

 Consider expanding this trial approach, especially in stores with similar customer demographics and sales patterns to Store 1.  
 
 The effect is consistent across multiple control comparisons, which reinforces the reliability of these findings.
